# FastF1 Telemetry

### This file contains scripts that serve as ETL footprints for the VisionF1 telemetry data pipeline. They obtain data from the API or the local cache, process it as needed, and the objective is to then upload it to the VisionF1 database (MongoDB Atlas).

## Race Pace

All data from 2018 to now.

In [ ]:
import fastf1
import pandas as pd
import numpy as np
from fastf1 import plotting
from datetime import datetime

SEASONS = range(2018, datetime.now().year + 1)

all_race_pace = []

for season in SEASONS:
    schedule = fastf1.get_event_schedule(season)
    schedule = schedule[schedule["EventFormat"] != "testing"]

    for _, event in schedule.iterrows():
        event_name = event["EventName"]
        round_number = event["RoundNumber"]
        try:
            session = fastf1.get_session(season, event_name, 'R')
            session.load()
        except Exception as e:
            print(f"Error loading session {season} {event_name}.")
            continue

        if not hasattr(session, '_laps'):
            print(f"No lap data for {season} {event_name}, skipping.")
            continue
        
        laps = session.laps
        laps = laps[laps['IsAccurate']]
        drivers = laps['Driver'].unique()

        driver_color_mapping = plotting.get_driver_color_mapping(session=session)

        for driver in drivers:
            filtered_laps = laps.pick_drivers([driver])
            
            avg_laptime = filtered_laps["LapTime"].mean()
            std_laptime = filtered_laps["LapTime"].std()

            driver_info = session.get_driver(driver)
            driver_first_name = driver_info.get("FirstName", driver)
            driver_last_name = driver_info.get("LastName", "")
            driver_position = driver_info.get("Position", None)
            driver_color = driver_color_mapping.get(driver, "#cccccc")
            team = driver_info.get("TeamId", None)
            team_name = driver_info.get("TeamName", None)
            team_color = driver_info.get("TeamColor", "#cccccc")

            all_race_pace.append({
                "season": season,
                "round": int(round_number),
                "event": event_name,
                "driver": driver,
                "driver_first_name": driver_first_name,
                "driver_last_name": driver_last_name,
                "driver_position": driver_position,
                "driver_color": driver_color,
                "team": team,
                "team_name": team_name,
                "team_color": team_color,
                "avg_laptime": avg_laptime.total_seconds(),
                "std_laptime": std_laptime,
                "race_pace_id": f"{season}_{round_number}_{driver}"
            })

df_race_pace = pd.DataFrame(all_race_pace)

df_race_pace = df_race_pace.replace({np.nan: None})

df_race_pace["race_pace_position"] = (
    df_race_pace.groupby(["season", "round"])["avg_laptime"]
    .rank(method="min", ascending=True)
    .astype(int)
)

In [96]:
df_race_pace

,season,round,event,driver,driver_first_name,driver_last_name,driver_position,driver_color,team,team_name,team_color,avg_laptime,std_laptime,race_pace_id,race_pace_position
0,2025,1,Australian Grand Prix,VER,Max,Verstappen,2.0,#0600ef,red_bull,Red Bull Racing,3671C6,90.314777,2.618743,2025_1_VER,3
1,2025,1,Australian Grand Prix,GAS,Pierre,Gasly,11.0,#ff87bc,alpine,Alpine,0093CC,92.140342,2.566940,2025_1_GAS,9
2,2025,1,Australian Grand Prix,ANT,Andrea Kimi,Antonelli,4.0,#27f4d2,mercedes,Mercedes,27F4D2,91.734382,2.871252,2025_1_ANT,6
3,2025,1,Australian Grand Prix,ALO,Fernando,Alonso,17.0,#00665f,aston_martin,Aston Martin,229971,92.502320,2.265482,2025_1_ALO,12
4,2025,1,Australian Grand Prix,LEC,Charles,Leclerc,8.0,#e80020,ferrari,Ferrari,E80020,91.704388,2.618302,2025_1_LEC,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2025,19,United States Grand Prix,SAI,Carlos,Sainz,20.0,#00a0dd,williams,Williams,1868DB,100.767250,0.411036,2025_19_SAI,17
361,2025,19,United States Grand Prix,HAD,Isack,Hadjar,16.0,#fcd700,rb,Racing Bulls,6C98FF,100.788254,1.247889,2025_19_HAD,18
362,2025,19,United States Grand Prix,RUS,George,Russell,6.0,#27f4d2,mercedes,Mercedes,00D7B6,99.807431,0.897683,2025_19_RUS,6
363,2025,19,United States Grand Prix,PIA,Oscar,Piastri,5.0,#ff8000,mclaren,McLaren,F47600,99.695700,0.823934,2025_19_PIA,5


In [97]:
ver = df_race_pace[ df_race_pace["driver"] == "VER" ]
ver_2025 = ver[ ver["season"] == 2025 ]
ver_2025

,season,round,event,driver,driver_first_name,driver_last_name,driver_position,driver_color,team,team_name,team_color,avg_laptime,std_laptime,race_pace_id,race_pace_position
0,2025,1,Australian Grand Prix,VER,Max,Verstappen,2.0,#0600ef,red_bull,Red Bull Racing,3671C6,90.314777,2.618743,2025_1_VER,3
20,2025,2,Chinese Grand Prix,VER,Max,Verstappen,4.0,#0600ef,red_bull,Red Bull Racing,3671C6,97.199452,1.050277,2025_2_VER,5
37,2025,3,Japanese Grand Prix,VER,Max,Verstappen,1.0,#0600ef,red_bull,Red Bull Racing,3671C6,92.459200,0.927919,2025_3_VER,2
62,2025,4,Bahrain Grand Prix,VER,Max,Verstappen,6.0,#0600ef,red_bull,Red Bull Racing,3671C6,98.083375,0.960517,2025_4_VER,5
78,2025,5,Saudi Arabian Grand Prix,VER,Max,Verstappen,2.0,#0600ef,red_bull,Red Bull Racing,3671C6,93.191066,0.582681,2025_5_VER,1
95,2025,6,Miami Grand Prix,VER,Max,Verstappen,4.0,#0600ef,red_bull,Red Bull Racing,3671C6,91.751291,1.136319,2025_6_VER,3
114,2025,7,Emilia Romagna Grand Prix,VER,Max,Verstappen,1.0,#0600ef,red_bull,Red Bull Racing,3671C6,80.305176,1.168307,2025_7_VER,1
134,2025,8,Monaco Grand Prix,VER,Max,Verstappen,4.0,#0600ef,red_bull,Red Bull Racing,4781D7,75.852114,1.453537,2025_8_VER,4
154,2025,9,Spanish Grand Prix,VER,Max,Verstappen,10.0,#0600ef,red_bull,Red Bull Racing,4781D7,79.654830,1.071211,2025_9_VER,1
173,2025,10,Canadian Grand Prix,VER,Max,Verstappen,2.0,#0600ef,red_bull,Red Bull Racing,4781D7,75.547278,0.688125,2025_10_VER,2


In [98]:
race_pace_2025 = df_race_pace[ df_race_pace["season"] == 2025 ]
race_pace_2025_15 = race_pace_2025[ race_pace_2025["round"] == 15 ]
race_pace_2025_15 = race_pace_2025_15.sort_values('avg_laptime')
dict = race_pace_2025_15.to_dict(orient="records")
print(dict)

[{'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'PIA', 'driver_first_name': 'Oscar', 'driver_last_name': 'Piastri', 'driver_position': 1.0, 'driver_color': '#ff8000', 'team': 'mclaren', 'team_name': 'McLaren', 'team_color': 'F47600', 'avg_laptime': 74.48166, 'std_laptime': 0.98182, 'race_pace_id': '2025_15_PIA', 'race_pace_position': 1}, {'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'NOR', 'driver_first_name': 'Lando', 'driver_last_name': 'Norris', 'driver_position': 18.0, 'driver_color': '#ff8000', 'team': 'mclaren', 'team_name': 'McLaren', 'team_color': 'F47600', 'avg_laptime': 74.665288, 'std_laptime': 0.920907, 'race_pace_id': '2025_15_NOR', 'race_pace_position': 2}, {'season': 2025, 'round': 15, 'event': 'Dutch Grand Prix', 'driver': 'VER', 'driver_first_name': 'Max', 'driver_last_name': 'Verstappen', 'driver_position': 2.0, 'driver_color': '#0600ef', 'team': 'red_bull', 'team_name': 'Red Bull Racing', 'team_color': '4781D7', 'avg_lapti

## Available Events

All available events in FastF1 from 2018 to now.

In [ ]:
import fastf1
import pandas as pd
from fastf1.ergast import Ergast

from utils import get_country_alpha2_code

ergast = Ergast()

SEASONS = range(2018, datetime.now().year + 1)

all_events = []
for season in SEASONS:
    schedule = fastf1.get_event_schedule(season)
    schedule = schedule[schedule["EventFormat"] != "testing"]
    
    for _, event in schedule.iterrows():
        event_name = event["EventName"]
        round_number = event["RoundNumber"]
        location = event["Location"]
        country = event["Country"]
        country_code = get_country_alpha2_code(country)
        event_format = event["EventFormat"]
        event_date = event["EventDate"]

        circuit = ergast.get_circuits(season=season, round=round_number)
        if not circuit.empty:
            circuit_id = circuit.iloc[0]["circuitId"]
            circuit_name = circuit.iloc[0]["circuitName"]
        else:
            print(f"Missing circuit information for {season} {round_number}")

        try:
            session = fastf1.get_session(season, event_name, 'R')
            session.load(telemetry=False, weather=False)
        except Exception as e:
            print(f"Error loading session {season} {event_name}.")
            continue

        results = session.results

        driver_codes = results["Abbreviation"].dropna().unique().tolist()
        driver_names = results["FullName"].dropna().unique().tolist()
        team_codes = results["TeamId"].dropna().unique().tolist()
        team_names = results["TeamName"].dropna().unique().tolist()
        team_colors = results["TeamColor"].dropna().unique().tolist()

        winner = results[results["Position"] == 1].iloc[0]["Abbreviation"] if not results[results["Position"] == 1].empty else None
        pole = results[results["GridPosition"] == 1].iloc[0]["Abbreviation"] if not results[results["GridPosition"] == 1].empty else None

        event_status = "ended" if winner else "upcoming"

        all_events.append({
            "season": season,
            "round": round_number,
            "event_name": event_name,
            "location": location,
            "country": country,
            "country_code": country_code,
            "event_date": event_date,
            "event_format": event_format,
            "event_status": event_status,
            "circuit_id": circuit_id,
            "circuit_name": circuit_name,
            "n_drivers": len(driver_codes),
            "driver_codes": driver_codes,
            "driver_names": driver_names,
            "n_teams": len(team_codes),
            "team_codes": team_codes,
            "team_names": team_names,
            "team_colors": team_colors,
            "winner": winner,
            "pole": pole,
            "event_id": f"{season}_{round_number}_{event_name}"
        })

df_events = pd.DataFrame(all_events)
df_events.sort_values(["season", "round"], inplace=True)
df_events.reset_index(drop=True, inplace=True)

df_events


core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']
core           INFO 	Loading data for Chinese Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data fo

,season,round,event_name,location,country,country_code,event_date,event_format,event_status,circuit_id,...,n_drivers,driver_codes,driver_names,n_teams,team_codes,team_names,team_colors,winner,pole,event_id
0,2025,1,Australian Grand Prix,Melbourne,Australia,AU,2025-03-16,conventional,ended,albert_park,...,20,"[NOR, VER, RUS, ANT, ALB, STR, HUL, LEC, PIA, ...","[Lando Norris, Max Verstappen, George Russell,...",10,"[mclaren, red_bull, mercedes, williams, aston_...","[McLaren, Red Bull Racing, Mercedes, Williams,...","[FF8000, 3671C6, 27F4D2, 64C4FF, 229971, 52E25...",NOR,NOR,2025_1_Australian Grand Prix
1,2025,2,Chinese Grand Prix,Shanghai,China,CN,2025-03-23,sprint_qualifying,ended,shanghai,...,20,"[PIA, NOR, RUS, VER, OCO, ANT, ALB, BEA, STR, ...","[Oscar Piastri, Lando Norris, George Russell, ...",10,"[mclaren, mercedes, red_bull, haas, williams, ...","[McLaren, Mercedes, Red Bull Racing, Haas F1 T...","[FF8000, 27F4D2, 3671C6, B6BABD, 64C4FF, 22997...",PIA,PIA,2025_2_Chinese Grand Prix
2,2025,3,Japanese Grand Prix,Suzuka,Japan,JP,2025-04-06,conventional,ended,suzuka,...,20,"[VER, NOR, PIA, LEC, RUS, ANT, HAM, HAD, ALB, ...","[Max Verstappen, Lando Norris, Oscar Piastri, ...",10,"[red_bull, mclaren, ferrari, mercedes, rb, wil...","[Red Bull Racing, McLaren, Ferrari, Mercedes, ...","[3671C6, FF8000, E80020, 27F4D2, 6692FF, 64C4F...",VER,VER,2025_3_Japanese Grand Prix
3,2025,4,Bahrain Grand Prix,Sakhir,Bahrain,BH,2025-04-13,conventional,ended,bahrain,...,20,"[PIA, RUS, NOR, LEC, HAM, VER, GAS, OCO, TSU, ...","[Oscar Piastri, George Russell, Lando Norris, ...",10,"[mclaren, mercedes, ferrari, red_bull, alpine,...","[McLaren, Mercedes, Ferrari, Red Bull Racing, ...","[FF8000, 27F4D2, E80020, 3671C6, 0093CC, B6BAB...",PIA,PIA,2025_4_Bahrain Grand Prix
4,2025,5,Saudi Arabian Grand Prix,Jeddah,Saudi Arabia,SA,2025-04-20,conventional,ended,jeddah,...,20,"[PIA, VER, LEC, NOR, RUS, ANT, HAM, SAI, ALB, ...","[Oscar Piastri, Max Verstappen, Charles Lecler...",10,"[mclaren, red_bull, ferrari, mercedes, william...","[McLaren, Red Bull Racing, Ferrari, Mercedes, ...","[FF8000, 3671C6, E80020, 27F4D2, 64C4FF, 6692F...",PIA,VER,2025_5_Saudi Arabian Grand Prix
5,2025,6,Miami Grand Prix,Miami Gardens,United States,US,2025-05-04,sprint_qualifying,ended,miami,...,20,"[PIA, NOR, RUS, VER, ALB, ANT, LEC, HAM, SAI, ...","[Oscar Piastri, Lando Norris, George Russell, ...",10,"[mclaren, mercedes, red_bull, williams, ferrar...","[McLaren, Mercedes, Red Bull Racing, Williams,...","[FF8000, 27F4D2, 3671C6, 64C4FF, E80020, 6692F...",PIA,VER,2025_6_Miami Grand Prix
6,2025,7,Emilia Romagna Grand Prix,Imola,Italy,IT,2025-05-18,conventional,ended,imola,...,20,"[VER, NOR, PIA, HAM, ALB, LEC, RUS, SAI, HAD, ...","[Max Verstappen, Lando Norris, Oscar Piastri, ...",10,"[red_bull, mclaren, ferrari, williams, mercede...","[Red Bull Racing, McLaren, Ferrari, Williams, ...","[3671C6, FF8000, E80020, 64C4FF, 27F4D2, 6692F...",VER,PIA,2025_7_Emilia Romagna Grand Prix
7,2025,8,Monaco Grand Prix,Monaco,Monaco,MC,2025-05-25,conventional,ended,monaco,...,20,"[NOR, LEC, PIA, VER, HAM, HAD, OCO, LAW, ALB, ...","[Lando Norris, Charles Leclerc, Oscar Piastri,...",10,"[mclaren, ferrari, red_bull, rb, haas, william...","[McLaren, Ferrari, Red Bull Racing, Racing Bul...","[F47600, ED1131, 4781D7, 6C98FF, 9C9FA2, 1868D...",NOR,NOR,2025_8_Monaco Grand Prix
8,2025,9,Spanish Grand Prix,Barcelona,Spain,ES,2025-06-01,conventional,ended,catalunya,...,19,"[PIA, NOR, LEC, RUS, HUL, HAM, HAD, GAS, ALO, ...","[Oscar Piastri, Lando Norris, Charles Leclerc,...",10,"[mclaren, ferrari, mercedes, sauber, rb, alpin...","[McLaren, Ferrari, Mercedes, Kick Sauber, Raci...","[F47600, ED1131, 00D7B6, 01C00E, 6C98FF, 00A1E...",PIA,PIA,2025_9_Spanish Grand Prix
9,2025,10,Canadian Grand Prix,Montréal,Canada,CA,2025-06-15,conventional,ended,villeneuve,...,20,"[RUS, VER, ANT, PIA, LEC, HAM, ALO, HUL, OCO, ...","[George Russell, Max Verstappen, Kimi Antonell...",10,"[mercedes, red_bull, mclaren, ferrari, as